In [10]:
pip install MetaTrader5

In [139]:
import pandas as pd
import MetaTrader5 as mt

import time
from datetime import datetime, time as dt_time

In [88]:
if not mt.initialize():
    print("Error: Failed to initialize MetaTrader.")

In [83]:
from config import login, password, server

if not mt.login(login, password, server):
    print("Error: Failed to login.")

Error: Failed to login.


In [124]:
account_info = mt.account_info()
print(account_info)

if not account_info:
    print("Error: Account not found.")
else:
    login_number = account_info.login
    balance = account_info.balance
    equity = account_info.equity

    print()
    print(f"Login Number: {login_number}")
    print(f"Balance: {balance}")
    print(f"Equity: {equity}")

AccountInfo(login=24936217, trade_mode=2, leverage=100, limit_orders=5000, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=True, balance=99.62, credit=0.0, profit=0.0, equity=99.62, margin=0.0, margin_free=99.62, margin_level=0.0, margin_so_call=120.0, margin_so_so=100.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Aidan Ouckama', server='Forex.com-Live 536', currency='USD', company='Gain Capital Group, LLC')

Login Number: 24936217
Balance: 99.62
Equity: 99.62


In [93]:
SYMBOL = "EURUSD"

In [125]:
mt.symbol_info(SYMBOL)._asdict()

{'custom': False,
 'chart_mode': 0,
 'select': True,
 'visible': True,
 'session_deals': 0,
 'session_buy_orders': 0,
 'session_sell_orders': 0,
 'volume': 0,
 'volumehigh': 0,
 'volumelow': 0,
 'time': 1724191237,
 'digits': 5,
 'spread': 14,
 'spread_float': True,
 'ticks_bookdepth': 0,
 'trade_calc_mode': 0,
 'trade_mode': 4,
 'start_time': 0,
 'expiration_time': 0,
 'trade_stops_level': 1,
 'trade_freeze_level': 0,
 'trade_exemode': 2,
 'swap_mode': 0,
 'swap_rollover3days': 3,
 'margin_hedged_use_leg': True,
 'expiration_mode': 15,
 'filling_mode': 1,
 'order_mode': 119,
 'order_gtc_mode': 0,
 'option_mode': 0,
 'option_right': 0,
 'bid': 1.1118999999999999,
 'bidhigh': 1.11218,
 'bidlow': 1.10712,
 'ask': 1.11204,
 'askhigh': 1.11234,
 'asklow': 1.10727,
 'last': 0.0,
 'lasthigh': 0.0,
 'lastlow': 0.0,
 'volume_real': 0.0,
 'volumehigh_real': 0.0,
 'volumelow_real': 0.0,
 'option_strike': 0.0,
 'point': 1e-05,
 'trade_tick_value': 1.0,
 'trade_tick_value_profit': 1.0,
 'trade_tic

In [126]:
ohlc = pd.DataFrame(mt.copy_rates_range(SYMBOL, mt.TIMEFRAME_M1, datetime(2024, 8, 20), datetime.now()))
ohlc

,time,open,high,low,close,tick_volume,spread,real_volume
0,1724126400,1.10811,1.10815,1.10807,1.10814,58,17,0
1,1724126460,1.10814,1.10827,1.10811,1.10827,80,17,0
2,1724126520,1.10826,1.10845,1.10826,1.10845,72,17,0
3,1724126580,1.10846,1.10848,1.10840,1.10845,64,17,0
4,1724126640,1.10844,1.10859,1.10842,1.10857,57,17,0
...,...,...,...,...,...,...,...,...
896,1724180160,1.11050,1.11059,1.11041,1.11054,54,17,0
897,1724180220,1.11053,1.11054,1.11042,1.11042,42,17,0
898,1724180280,1.11046,1.11055,1.11042,1.11049,93,18,0
899,1724180340,1.11049,1.11050,1.11034,1.11043,94,18,0


In [127]:
ohlc['time']  = pd.to_datetime(ohlc['time'], unit='s')
ohlc

,time,open,high,low,close,tick_volume,spread,real_volume
0,2024-08-20 04:00:00,1.10811,1.10815,1.10807,1.10814,58,17,0
1,2024-08-20 04:01:00,1.10814,1.10827,1.10811,1.10827,80,17,0
2,2024-08-20 04:02:00,1.10826,1.10845,1.10826,1.10845,72,17,0
3,2024-08-20 04:03:00,1.10846,1.10848,1.10840,1.10845,64,17,0
4,2024-08-20 04:04:00,1.10844,1.10859,1.10842,1.10857,57,17,0
...,...,...,...,...,...,...,...,...
896,2024-08-20 18:56:00,1.11050,1.11059,1.11041,1.11054,54,17,0
897,2024-08-20 18:57:00,1.11053,1.11054,1.11042,1.11042,42,17,0
898,2024-08-20 18:58:00,1.11046,1.11055,1.11042,1.11049,93,18,0
899,2024-08-20 18:59:00,1.11049,1.11050,1.11034,1.11043,94,18,0


In [100]:
import plotly.express as px

fig = px.line(ohlc, x = ohlc['time'], y = ohlc['close'])
fig.show()

In [115]:
# order request examples
volume = 0.01

# BUY position
buy_take_profit = 1.11175
buy_stop_loss = 1.11150
buy_request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume, # FLOAT
    "type": mt.ORDER_TYPE_BUY,
    "price": mt.symbol_info_tick(SYMBOL).ask,
    "sl": buy_stop_loss, # FLOAT
    "tp": buy_take_profit, # FLOAT
    "comment": "Buy Position Opened.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

# SELL position
sell_stop_loss = 1.11175
sell_take_profit = 1.11150
sell_request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume,
    "type": mt.ORDER_TYPE_SELL,
    "price": mt.symbol_info_tick(SYMBOL).bid,
    "sl": sell_stop_loss,
    "tp": sell_take_profit,
    "comment": "Sell Position Opened.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

In [116]:
result = mt.order_send(buy_request)
result

position_id = result._asdict()['order']

In [117]:
# order closing examples

# BUY order closing
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume, # FLOAT
    "type": mt.ORDER_TYPE_SELL,
    "position": position_id, # id of the position we want to close
    "comment": "Buy Position Closed.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

# SELL order closing
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume, # FLOAT
    "type": mt.ORDER_TYPE_BUY,
    "position": position_id, # id of the position we want to close
    "comment": "Sell Position Closed.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

close = mt.order_send(request)
print(close)

OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=0, type_time=0, expiration=0, comment='Buy Position Closed.', position=6527693, position_by=0))


In [138]:
today = datetime.today().date()
today_at_midnight = datetime.combine(today, time.min)

today_at_midnight

AttributeError: module 'time' has no attribute 'min'

In [143]:
# trading bot

SYMBOL = "EURUSD"
QTY = 0.01
ITERATIONS = 100

SL_PCT = 0.05
TP_PCT = 0.1

BUY_TYPE = 1
SELL_TYPE = 0

BUY_ORDER_TYPE = mt.ORDER_TYPE_BUY
SELL_ORDER_TYPE = mt.ORDER_TYPE_SELL

def generate_ohlc():
    today = datetime.today().date()
    today_at_midnight = datetime.combine(today, dt_time.min)
    
    ohlc = pd.DataFrame(mt.copy_rates_range(SYMBOL, mt.TIMEFRAME_M1, today_at_midnight, datetime.now()))
    ohlc['time']  = pd.to_datetime(ohlc['time'], unit='s')
    return ohlc

def place_order(symbol, qty, order_type, price, sl, tp):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": qty,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp, 
        "comment": "Position Opened.",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK, 
    }
    
    order = mt.order_send(request)
    return order

def close_order(symbol, qty, order_type, price):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": qty,
        "type": order_type,
        "position": mt.positions_get()[0]._asdict()['ticket'],
        "price": price,
        "comment": "Position Opened.",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }
    
    close = mt.order_send(request)
    return close

In [ ]:
# strategy
for i in range(ITERATIONS):
    ohlc = generate_ohlc()
    print(ohlc)
    
    buy_price = mt.symbol_info_tick(SYMBOL).ask
    sell_price = mt.symbol_info_tick(SYMBOL).bid

    buy_sl = buy_price * (1 - SL_PCT)
    buy_tp = buy_price * (1 + TP_PCT)
    sell_sl = sell_price * (1 + SL_PCT)
    sell_tp = sell_price * (1 - TP_PCT)

    current_close = list(ohlc[-1:]['close'])[0]
    last_close = list(ohlc[-2:]['close'])[0]
    last_high = list(ohlc[-2:]['high'])[0]
    last_low = list(ohlc[-2:]['low'])[0]

    long_condition = current_close > last_high
    short_condition = current_close < last_low
    closelong_condition = current_close < last_close
    closeshort_condition = current_close > last_close

    already_sell = False
    already_buy = False
    try:
        already_sell = mt.positions_get()[0]._asdict()['type'] == BUY_TYPE
        already_buy = mt.positions_get()[0]._asdict()['type'] == SELL_TYPE
    except:
        pass

    no_positions = len(mt.positions_get()) == 0

    if long_condition:
        if no_positions:
            place_order(SYMBOL, QTY, BUY_ORDER_TYPE, buy_price, buy_sl, buy_tp)
            print("Buy Order Placed.")
        if already_sell:
            close_order(SYMBOL, QTY, BUY_ORDER_TYPE, buy_price)
            print("Sell Position Closed.")
            time.sleep(1)
            place_order(SYMBOL, QTY, BUY_ORDER_TYPE, buy_price, buy_sl, buy_tp)

    if short_condition:
        if no_positions:
            place_order(SYMBOL, QTY, SELL_ORDER_TYPE, sell_price, sell_sl, sell_tp)
            print("Sell Order Placed.")
        if already_buy:
            close_order(SYMBOL, QTY, SELL_ORDER_TYPE, sell_price)
            print("Buy Position Closed.")
            time.sleep(1)
            place_order(SYMBOL, QTY, SELL_ORDER_TYPE, sell_price, sell_sl, sell_tp)
            print("Sell Order Placed.")
    
    if closelong_condition and already_buy:
        close_order(SYMBOL, QTY, SELL_ORDER_TYPE, sell_price)
        print("Buy Position Closed.")
    if closeshort_condition and already_sell:
        close_order(SYMBOL, QTY, BUY_ORDER_TYPE, buy_price)
        print("Sell Position Closed.")
        
    time.sleep(60)

                   time     open     high      low    close  tick_volume  \
0   2024-08-20 04:00:00  1.10811  1.10815  1.10807  1.10814           58   
1   2024-08-20 04:01:00  1.10814  1.10827  1.10811  1.10827           80   
2   2024-08-20 04:02:00  1.10826  1.10845  1.10826  1.10845           72   
3   2024-08-20 04:03:00  1.10846  1.10848  1.10840  1.10845           64   
4   2024-08-20 04:04:00  1.10844  1.10859  1.10842  1.10857           57   
..                  ...      ...      ...      ...      ...          ...   
904 2024-08-20 19:04:00  1.11042  1.11061  1.11041  1.11060           69   
905 2024-08-20 19:05:00  1.11061  1.11085  1.11060  1.11085           70   
906 2024-08-20 19:06:00  1.11082  1.11086  1.11081  1.11085           67   
907 2024-08-20 19:07:00  1.11086  1.11103  1.11085  1.11103           65   
908 2024-08-20 19:08:00  1.11102  1.11115  1.11102  1.11115           72   

     spread  real_volume  
0        17            0  
1        17            0  
2     

                   time     open     high      low    close  tick_volume  \
0   2024-08-20 04:00:00  1.10811  1.10815  1.10807  1.10814           58   
1   2024-08-20 04:01:00  1.10814  1.10827  1.10811  1.10827           80   
2   2024-08-20 04:02:00  1.10826  1.10845  1.10826  1.10845           72   
3   2024-08-20 04:03:00  1.10846  1.10848  1.10840  1.10845           64   
4   2024-08-20 04:04:00  1.10844  1.10859  1.10842  1.10857           57   
..                  ...      ...      ...      ...      ...          ...   
911 2024-08-20 19:11:00  1.11130  1.11131  1.11098  1.11101           90   
912 2024-08-20 19:12:00  1.11103  1.11121  1.11103  1.11120           59   
913 2024-08-20 19:13:00  1.11121  1.11136  1.11121  1.11129           59   
914 2024-08-20 19:14:00  1.11131  1.11142  1.11131  1.11137           42   
915 2024-08-20 19:15:00  1.11137  1.11137  1.11120  1.11124           65   

     spread  real_volume  
0        17            0  
1        17            0  
2     